# Installations

In [2]:
import sys
import os
import torch
import random
import numpy as np
from tqdm import tqdm
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import pdb

sys.path.append('/home/roy206/StrGNN/pytorch_DGCNN')
from DGCNN_embedding import DGCNN
from mlp_dropout import MLPClassifier, MLPRegression
from sklearn import metrics
from util import cmd_args, load_data
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from torch_geometric.utils.convert import to_networkx, from_networkx
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
from GADNR import *

====== begin of gnn configuration ======
| msg_average = 0
======   end of gnn configuration ======


# Main

In [5]:
import torch
import numpy as np
import sys, copy, math, time, pdb
import pickle as pickle
import scipy.io as sio
import scipy.sparse as ssp
import os
import os.path
import random
import argparse
import pickle
sys.path.append('/home/roy206/StrGNN/pytorch_DGCNN/')
from main import *
from util_functions import *
from os import path

sys.path.append('/home/roy206/StrGNN/')


def loop_dataset(g_list, GADNR, sample_idxes, optimizer=None, bsize=cmd_args.batch_size):
    total_loss = []
    total_iters = (len(sample_idxes) + (bsize - 1) * (optimizer is None)) // bsize
    pbar = tqdm(list(range(total_iters)), unit='batch')
    all_targets = []
    all_scores = []
    
    best_auc = 0
    bestc_auc = 0
    bests_auc = 0
    bestj_auc = 0
    bestsj_auc = 0
    

    n_samples = 0
    for pos in range(total_iters):
        selected_idx = sample_idxes[pos * bsize : (pos + 1) * bsize]
        
        batch_graph = [g_list[idx] for idx in selected_idx]
        targets = [g_list[idx][-1].label for idx in selected_idx]
        all_targets += targets
        
        # print(len(batch_graph))
        # print(len(all_targets))
        
        y_list = []
        yc_list = []
        ys_list = []
        yj_list = []
        ysj_list = []
        
        for graphs in batch_graph:
            # print(len(graphs))
            for graph in graphs:
                
                if  graph.num_nodes>5 and graph.num_edges > 5:
                    
                    edgelist = graph.edge_pairs
                    total_edges = len(edgelist)//2
                    
                    edge_pairs = []
                    
                    for idx in range(total_edges):
                        x = edgelist[idx]
                        y = edgelist[idx+total_edges]
                        edge_pairs.append((x,y))
                    
                    G = nx.from_edgelist(edge_pairs)
                    
                    n_param = max(1, G.number_of_nodes() // 20)
                    m_param = max(1,(total_edges // G.number_of_nodes()) )
                    
                    G = from_networkx(G)
                    num_nodes = G.num_nodes
                    node_feat = torch.rand(num_nodes, 64)                    
                    G.x = node_feat
                    
                    G, yc = gen_contextual_outlier(G,n_param,m_param)
                    G, ys = gen_structural_outlier(G,n_param,m_param)
                    G, yj = gen_joint_structural_outliers(G,n_param,m_param)
                    
                    ysj = torch.logical_or(ys, yj).int()
                    y = torch.logical_or(yc, ys).int()
                    y = torch.logical_or(y, yj).int()
                    
                    in_nodes = G.edge_index[0,:]
                    out_nodes = G.edge_index[1,:]

                    neighbor_dict = {}
                    for in_node, out_node in zip(in_nodes, out_nodes):
                        if in_node.item() not in neighbor_dict:
                            neighbor_dict[in_node.item()] = []
                        neighbor_dict[in_node.item()].append(out_node.item())

                    neighbor_num_list = []
                    for i in neighbor_dict:
                        neighbor_num_list.append(len(neighbor_dict[i]))

                    neighbor_num_list = torch.tensor(neighbor_num_list).to(device)
                    
                    GADNR.num_nodes = num_nodes
                    GADNR.m_batched = torch.distributions.Normal(torch.zeros(sample_size, num_nodes, hidden_dim),
                                            torch.ones(sample_size, num_nodes, hidden_dim))
                    
                    GADNR.neighbor_num_list = neighbor_num_list
                    GADNR.std_agg = PNAConv(hidden_dim, hidden_dim, aggregators=["std"],scalers=["identity"], 
                        deg=GADNR.neighbor_num_list)
                    
                    G = G.to(device)
                    GADNR = GADNR.to(device)
                    
                    loss,loss_per_node,h_loss,degree_loss,feature_loss = GADNR(G.edge_index, G.x, 
                    neighbor_num_list, neighbor_dict, device=device)
                    
                    loss_per_node = loss_per_node.cpu().detach()
        
                    h_loss = h_loss.cpu().detach()
                    degree_loss = degree_loss.cpu().detach()
                    feature_loss = feature_loss.cpu().detach()

                    # h_loss_norm = h_loss / (torch.max(h_loss) - torch.min(h_loss))
                    # degree_loss_norm = degree_loss / (torch.max(degree_loss) - torch.min(degree_loss))
                    # feature_loss_norm = feature_loss / (torch.max(feature_loss) - torch.min(feature_loss))
                    
                    h_loss_weight = 1e-2
                    degree_loss_weight = 0.5
                    feature_loss_weight = 0.8
                    
                    if torch.isfinite(h_loss).all() and torch.isfinite(degree_loss).all() and torch.isfinite(feature_loss).all():

                        comp_loss = h_loss_weight * h_loss + degree_loss_weight *  degree_loss 
                        + feature_loss_weight * feature_loss

                        # print("neighbor num: ", neighbor_num_list)
                        # print("degree loss: ", degree_loss)

                        auc_score = eval_roc_auc(y.numpy(), comp_loss.numpy()) * 100
                        # print("Dataset Name: ",args.data_name, ", AUC Score(benchmark/combined): ", auc_score)

                        contextual_auc_score = eval_roc_auc(yc.numpy(), comp_loss.numpy()) * 100
                        # print("Dataset Name: ",args.data_name, ", AUC Score (contextual): ", contextual_auc_score)

                        dense_structural_auc_score = eval_roc_auc(ys.numpy(), comp_loss.numpy()) * 100
                        # print("Dataset Name: ",args.data_name, ", AUC Score (structural): ", dense_structural_auc_score)

                        joint_structural_auc_score = eval_roc_auc(yj.numpy(), comp_loss.numpy()) * 100
                        # print("Dataset Name: ",args.data_name, ", AUC Score (joint-type): ", joint_structural_auc_score)

                        structure_type_auc_score = eval_roc_auc(ysj.numpy(), comp_loss.numpy()) * 100
                        # print("Dataset Name: ",args.data_name, ", AUC Score (structure type): ", joint_structural_auc_score) 

                        y_list.append(auc_score)
                        yc_list.append(contextual_auc_score)
                        ys_list.append(dense_structural_auc_score)
                        yj_list.append(joint_structural_auc_score)
                        ysj_list.append(structure_type_auc_score)
                    
        auc = sum(y_list)/len(y_list)
        c_auc = sum(yc_list)/len(yc_list)
        s_auc = sum(ys_list)/len(ys_list)
        j_auc = sum(yj_list)/len(yj_list)
        sj_auc = sum(ysj_list)/len(ysj_list)
        
        best_auc = max(best_auc, auc)
        bestc_auc = max(bestc_auc, c_auc)
        bests_auc = max(bests_auc, s_auc)
        bestj_auc = max(bestj_auc, j_auc)
        bestsj_auc = max(bestsj_auc, sj_auc)
        
        print("Dataset Name: ",args.data_name, ", AUC Score(benchmark/combined): ", best_auc)
        print("Dataset Name: ",args.data_name, ", AUC Score(contextual): ", bestc_auc)
        print("Dataset Name: ",args.data_name, ", AUC Score(structural): ", bests_auc)
        print("Dataset Name: ",args.data_name, ", AUC Score(joint): ", bestj_auc)
        print("Dataset Name: ",args.data_name, ", AUC Score(joint_structural): ", bestsj_auc)
        
        # import sys
        # sys.exit(0)
        
        if classifier.regression:
            pred, mae, loss = classifier(batch_graph)
            all_scores.append(pred.cpu().detach())  # for binary classification
        else:
            logits, loss, acc = classifier(batch_graph)
            all_scores.append(logits[:, 1].cpu().detach())  # for binary classification

        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss = loss.data.cpu().detach().numpy()
        if classifier.regression:
            pbar.set_description('MSE_loss: %0.5f MAE_loss: %0.5f' % (loss, mae) )
            total_loss.append( np.array([loss, mae]) * len(selected_idx))
        else:
            pbar.set_description('loss: %0.5f acc: %0.5f' % (loss, acc) )
            total_loss.append( np.array([loss, acc]) * len(selected_idx))


        n_samples += len(selected_idx)
    if optimizer is None:
        assert n_samples == len(sample_idxes)
    total_loss = np.array(total_loss)
    avg_loss = np.sum(total_loss, 0) / n_samples
    all_scores = torch.cat(all_scores).cpu().numpy()
    
    # np.savetxt('test_scores.txt', all_scores)  # output test predictions
    
    if not classifier.regression:
        all_targets = np.array(all_targets)
        fpr, tpr, _ = metrics.roc_curve(all_targets, all_scores, pos_label=1)
        avg_precision = average_precision_score(all_targets, all_scores)
        precision, recall, _ = precision_recall_curve(all_targets, all_scores)
        auc = metrics.auc(fpr, tpr)
        pr_auc = metrics.auc(recall, precision)
        avg_loss = np.concatenate((avg_loss, [auc], [avg_precision], [pr_auc]))
    return avg_loss

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

parser = argparse.ArgumentParser()

# general settings
parser.add_argument('-f')
parser.add_argument('--data-name', default='USAir', help='network name')
parser.add_argument('--train-name', default=None, help='train name')
parser.add_argument('--test-name', default=None, help='test name')
parser.add_argument('--max-train-num', type=int, default=100000, 
                    help='set maximum number of train links (to fit into memory)')
parser.add_argument('--no-cuda', action='store_true', default=True,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=42, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--test-ratio', type=float, default=0.815,
                    help='ratio of test links')
parser.add_argument('--window', type=int, default=5,
                    help='window size')
parser.add_argument('--graph', default='acc_email.npy')
parser.add_argument('--split', default='email0.05')
parser.add_argument('--gpu', default='1', help='gpu number')
# model settings
parser.add_argument('--hop', default=1, metavar='S', 
                    help='enclosing subgraph hop number, \
                    options: 1, 2,..., "auto"')
parser.add_argument('--max-nodes-per-hop', default=None, 
                    help='if > 0, upper bound the # nodes per hop by subsampling')
parser.add_argument('--use-embedding', action='store_true', default=False,
                    help='whether to use node2vec node embeddings')
parser.add_argument('--use-attribute', action='store_true', default=False,
                    help='whether to use node attributes')
args = parser.parse_args()


os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
print(args)

random.seed(cmd_args.seed)
np.random.seed(cmd_args.seed)
torch.manual_seed(cmd_args.seed)
torch.cuda.manual_seed(cmd_args.seed)


if args.hop != 'auto':
    args.hop = int(args.hop)
if args.max_nodes_per_hop is not None:
    args.max_nodes_per_hop = int(args.max_nodes_per_hop)


'''Prepare data'''
args.file_dir = os.path.dirname(os.path.realpath('__file__'))
args.res_dir = os.path.join(args.file_dir, 'results/{}'.format(args.data_name))

if args.train_name is None:
    args.data_dir = os.path.join(args.file_dir, 'data/{}.mat'.format(args.data_name))
    
    net = np.load('data/'+args.graph, allow_pickle=True)
    # net = np.loadtxt('data/'+args.graph,  delimiter=',')
    

    if False:
        net_ = net.toarray()
        assert(np.allclose(net_, net_.T, atol=1e-8))
    #Sample train and test links
    f = np.load('data/'+args.split+'.npz')
    # f = np.loadtxt('data/'+args.split+'.npz',  delimiter=',')
    
    train_pos_id, train_neg_id, test_pos_id, test_neg_id = f['train_pos_id'], f['train_neg_id'], f['test_pos_id'], f['test_neg_id']
    train_pos, train_neg, test_pos, test_neg = f['train_pos'], f['train_neg'], f['test_pos'], f['test_neg']
else:
    args.train_dir = os.path.join(args.file_dir, 'data/{}'.format(args.train_name))
    args.test_dir = os.path.join(args.file_dir, 'data/{}'.format(args.test_name))
    train_idx = np.loadtxt(args.train_dir, dtype=int)
    test_idx = np.loadtxt(args.test_dir, dtype=int)
    max_idx = max(np.max(train_idx), np.max(test_idx))
    net = ssp.csc_matrix((np.ones(len(train_idx)), (train_idx[:, 0], train_idx[:, 1])), shape=(max_idx+1, max_idx+1))
    net[train_idx[:, 1], train_idx[:, 0]] = 1  # add symmetric edges
    net[np.arange(max_idx+1), np.arange(max_idx+1)] = 0  # remove self-loops
    #Sample negative train and test links
    train_pos = (train_idx[:, 0], train_idx[:, 1])
    test_pos = (test_idx[:, 0], test_idx[:, 1])
    train_pos, train_neg, test_pos, test_neg = sample_dyn(net, train_pos=train_pos, test_pos=test_pos, max_train_num=args.max_train_num)


'''Train and apply classifier'''
A = net.copy()  # the observed network
# A[test_pos[0], test_pos[1]] = 0  # mask test links
# A[test_pos[1], test_pos[0]] = 0  # mask test links

node_information = None
if args.use_embedding:
    embeddings = generate_node2vec_embeddings(A, 128, True, train_neg)
    node_information = embeddings
if args.use_attribute and attributes is not None:
    if node_information is not None:
        node_information = np.concatenate([node_information, attributes], axis=1)
    else:
        node_information = attributes

if not path.exists('data/'+args.split+'h'+str(args.hop)):
    train_graphs, test_graphs, max_n_label = dyn_links2subgraphs(A, args.window, train_pos_id, train_pos, train_neg_id, train_neg, test_pos_id, test_pos, test_neg_id, test_neg, args.hop, args.max_nodes_per_hop, node_information)
    
    print(('# train: %d, # test: %d' % (len(train_graphs), len(test_graphs))))
    with open('data/'+args.split+'h'+str(args.hop), 'wb') as f:
        pickle.dump([train_graphs, test_graphs, max_n_label], f, protocol=4)
else:
    with open('data/'+args.split+'h'+str(args.hop), 'rb') as f:
        train_graphs, test_graphs, max_n_label = pickle.load(f)
        print(('# train: %d, # test: %d' % (len(train_graphs), len(test_graphs))))
        

# DGCNN configurations
cmd_args.gm = 'DGCNN'
cmd_args.sortpooling_k = 0.6
cmd_args.latent_dim = [32, 32, 32, 1]
cmd_args.hidden = 128
cmd_args.out_dim = 0
cmd_args.dropout = True
cmd_args.num_class = 2
cmd_args.mode = 'gpu'
cmd_args.num_epochs = 50
cmd_args.learning_rate = 1e-4
cmd_args.batch_size = 32
cmd_args.printAUC = True
cmd_args.feat_dim = max_n_label + 1
cmd_args.attr_dim = 0
cmd_args.window = 5
if node_information is not None:
    cmd_args.attr_dim = node_information.shape[1]
if cmd_args.sortpooling_k <= 1:
    A = []
    for i in train_graphs:
        #print(type(i[-1]))
        A.append(i[-1])
    for i in test_graphs:
        A.append(i[-1])   
    #print(type(A[0]))
    num_nodes_list = sorted([g.num_nodes for g in A])
    cmd_args.sortpooling_k = num_nodes_list[int(math.ceil(cmd_args.sortpooling_k * len(num_nodes_list))) - 1]
    cmd_args.sortpooling_k = max(10, cmd_args.sortpooling_k)
    print(('k used in SortPooling is: ' + str(cmd_args.sortpooling_k)))

in_dim = 64
hidden_dim = 64
sample_size = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = "GCN"
lambda_loss1 = 1e-2
lambda_loss2 = 0.5
lambda_loss3 = 0.8

    
GADNR = GNNStructEncoder(in_dim, hidden_dim, hidden_dim, 2, sample_size, device=device, GNN_name=encoder, 
                    lambda_loss1=lambda_loss1, lambda_loss2=lambda_loss2,lambda_loss3=lambda_loss3)
GADNR.cuda()    
    
    
classifier = Classifier()
if cmd_args.mode == 'gpu':
    classifier = classifier.cuda()

optimizer = optim.Adam(classifier.parameters(), lr=cmd_args.learning_rate)

train_idxes = list(range(len(train_graphs)))
best_loss = None
best_auc = 0
for epoch in range(cmd_args.num_epochs):
    random.shuffle(train_idxes)
    classifier.train()
    
    avg_loss = loop_dataset(train_graphs, GADNR, train_idxes, optimizer=optimizer)
    # print(avg_loss.shape)
    # import sys
    # sys.exit(0)
    if not cmd_args.printAUC:
        avg_loss[2] = 0.0
    print(('\033[92maverage training of epoch %d: loss %.5f acc %.5f auc %.5f\033[0m' % (epoch, avg_loss[0], avg_loss[1], avg_loss[2])))

    classifier.eval()
    test_loss = loop_dataset(test_graphs, GADNR, list(range(len(test_graphs))))
    if not cmd_args.printAUC:
        test_loss[2] = 0.0
    print(('\033[93maverage test of epoch %d: loss %.5f acc %.5f auc %.5f avg_precision %.5f precision-recall auc %.5f\033[0m' % (epoch, test_loss[0], test_loss[1], test_loss[2], test_loss[3], test_loss[4])))
    if test_loss[2] > best_auc:
        best_auc = test_loss[2]
print('best_auc = ', best_auc)

# with open('acc_results.txt', 'a+') as f:
#     f.write(str(test_loss[1]) + '\n')

# if cmd_args.printAUC:
#     with open('auc_results.txt', 'a+') as f:
#         f.write(str(test_loss[2]) + '\n')


Namespace(cuda=False, data_name='USAir', f='/home/roy206/.local/share/jupyter/runtime/kernel-5846faf1-cc39-4e0a-856f-ce281ccefd6c.json', gpu='1', graph='acc_email.npy', hop=1, max_nodes_per_hop=None, max_train_num=100000, no_cuda=True, seed=42, split='email0.05', test_name=None, test_ratio=0.815, train_name=None, use_attribute=False, use_embedding=False, window=5)
# train: 3092, # test: 1286
k used in SortPooling is: 43
Initializing DGCNN


  0%|          | 0/96 [00:00<?, ?batch/s]

Dataset Name:  USAir , AUC Score(benchmark/combined):  52.92332825478038
Dataset Name:  USAir , AUC Score(contextual):  50.72708919526866
Dataset Name:  USAir , AUC Score(structural):  59.093925221369254
Dataset Name:  USAir , AUC Score(joint):  49.232630410171794
Dataset Name:  USAir , AUC Score(joint_structural):  55.23010356944648


/home/roy206/StrGNN/pytorch_DGCNN/lib/gnn_lib.py:80: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:605.)
  n2n_sp = torch.sparse.FloatTensor(n2n_idxes, n2n_vals, torch.Size([total_num_nodes, total_num_nodes]))
loss: 0.68578 acc: 0.59375:   0%|          | 0/96 [00:11<?, ?batch/s]

Dataset Name:  USAir , AUC Score(benchmark/combined):  56.22408202445687
Dataset Name:  USAir , AUC Score(contextual):  53.27086496865437
Dataset Name:  USAir , AUC Score(structural):  59.093925221369254
Dataset Name:  USAir , AUC Score(joint):  52.228703529750256
Dataset Name:  USAir , AUC Score(joint_structural):  57.07743279848596


...

Dataset Name:  USAir , AUC Score(benchmark/combined):  58.9842939019181
Dataset Name:  USAir , AUC Score(contextual):  55.61735067735978
Dataset Name:  USAir , AUC Score(structural):  66.47974349500672
Dataset Name:  USAir , AUC Score(joint):  55.71485711122096
Dataset Name:  USAir , AUC Score(joint_structural):  62.83132783937522


loss: 0.35500 acc: 0.93750:   0%|          | 0/96 [01:15<?, ?batch/s]

Dataset Name:  USAir , AUC Score(benchmark/combined):  58.9842939019181
Dataset Name:  USAir , AUC Score(contextual):  55.61735067735978
Dataset Name:  USAir , AUC Score(structural):  66.47974349500672
Dataset Name:  USAir , AUC Score(joint):  55.71485711122096
Dataset Name:  USAir , AUC Score(joint_structural):  62.83132783937522


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/roy206/.conda/envs/cent7/2020.11-py38/py39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/roy206/.conda/envs/cent7/2020.11-py38/py39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/roy206/.conda/envs/cent7/2020.11-py38/py39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/roy206/.conda/envs/cent7/2020.11-py38/py39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/roy206/.conda/envs/cent7/2020.11-py38/py39/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/roy206/.conda/envs/cent

In [2]:
import torch
torch.cuda.is_available()

True

In [ ]:
# !ipython
!import torch
!torch.cuda.is_available()

In [ ]:
!source ~/venvs/py39/bin/activate
!which python

In [ ]:
!jupyter kernelspec uninstall py39

In [ ]:
!python3